In [1]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('/content/jupyter/mta-accessibility')
# from src.data import process_turnstile
from ast import literal_eval
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df_cleaned = pd.read_pickle('../../data/cache/cleaned_turnstile.pkl.gz')

In [3]:
df_cleaned[df_cleaned.UNIT == "R158"].head()

,datetime,C/A,DESC,DIVISION,ENTRIES,EXITS,LINENAME,SCP,STATION,UNIT,entry_diff,exit_diff,entry_diff_abs,exit_diff_abs,time_diffs
87992,2019-01-01 00:18:03,N335,REGULAR,IND,11826016,4487529,EF,01-00-01,KEW GARDENS,R158,25.0,20.0,25.0,20.0,2.800833
87993,2019-01-01 00:18:03,N335,REGULAR,IND,6296098,24797894,EF,01-00-04,KEW GARDENS,R158,19.0,97.0,19.0,97.0,2.800833
87994,2019-01-01 00:18:03,N335,REGULAR,IND,5109204,1660085,EF,01-06-02,KEW GARDENS,R158,10.0,5.0,10.0,5.0,2.800833
87995,2019-01-01 00:18:03,N335,REGULAR,IND,6313366,9707730,EF,01-00-02,KEW GARDENS,R158,10.0,45.0,10.0,45.0,2.800833
87996,2019-01-01 00:18:03,N335,REGULAR,IND,3706461,349900,EF,01-06-01,KEW GARDENS,R158,26.0,7.0,26.0,7.0,2.800833


In [4]:
filtered_turnstile = df_cleaned.groupby(['LINENAME', 'STATION', 'UNIT', 'datetime']).sum().reset_index()

In [24]:
def _interpolate(data):
    augmented = data.resample('1H').asfreq()
    augmented = pd.concat([augmented,data])
    augmented = augmented.sort_index(ascending=True)
#     print(augmented)
    augmented.entry_diff_abs.interpolate(method='linear', inplace=True)
    augmented.exit_diff_abs.interpolate(method='linear', inplace=True)
    augmented.fillna(method='ffill', inplace=True)
    return augmented

In [38]:
filtered_turnstile.set_index(pd.DatetimeIndex(filtered_turnstile.datetime), inplace=True)
sample_turnstile = filtered_turnstile[filtered_turnstile.UNIT == "R158"]

In [39]:
sample_turnstile.head()

,LINENAME,STATION,UNIT,datetime,ENTRIES,EXITS,entry_diff,exit_diff,entry_diff_abs,exit_diff_abs,time_diffs
datetime,,,,,,,,,,,
2019-01-01 00:18:03,EF,KEW GARDENS,R158,2019-01-01 00:18:03,58838769,67536224,184.0,333.0,184.0,333.0,25.2075
2019-01-01 00:19:32,EF,KEW GARDENS,R158,2019-01-01 00:19:32,58838773,67536224,4.0,0.0,4.0,0.0,0.2225
2019-01-01 00:20:59,EF,KEW GARDENS,R158,2019-01-01 00:20:59,58838773,67536224,0.0,0.0,0.0,0.0,0.2175
2019-01-01 00:22:23,EF,KEW GARDENS,R158,2019-01-01 00:22:23,58838773,67536228,0.0,4.0,0.0,4.0,0.2100
2019-01-01 00:43:09,EF,KEW GARDENS,R158,2019-01-01 00:43:09,58838778,67536239,5.0,11.0,5.0,11.0,3.1150


In [40]:
hourly_sampled = sample_turnstile.resample('1H').asfreq()
augmented = pd.concat([hourly_sampled,sample_turnstile])
augmented = augmented.sort_index(ascending=True)

In [41]:
augmented.entry_diff_abs.interpolate(method='linear', inplace=True)
augmented.exit_diff_abs.interpolate(method='linear', inplace=True)
augmented.fillna(method='ffill', inplace=True)
res = augmented.loc[hourly_sampled.index]

In [42]:
res.head(20)

,LINENAME,STATION,UNIT,datetime,ENTRIES,EXITS,entry_diff,exit_diff,entry_diff_abs,exit_diff_abs,time_diffs
datetime,,,,,,,,,,,
2019-01-01 00:00:00,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 01:00:00,EF,KEW GARDENS,R158,2019-01-01 00:43:09,58838778.0,67536239.0,5.0,11.0,230.50,386.0,3.1150
2019-01-01 02:00:00,EF,KEW GARDENS,R158,2019-01-01 01:30:00,108160437.0,95632468.0,456.0,761.0,356.75,643.0,51.0275
2019-01-01 03:00:00,EF,KEW GARDENS,R158,2019-01-01 01:30:00,108160437.0,95632468.0,456.0,761.0,257.50,525.0,51.0275
2019-01-01 04:00:00,EF,KEW GARDENS,R158,2019-01-01 01:30:00,108160437.0,95632468.0,456.0,761.0,158.25,407.0,51.0275
2019-01-01 05:00:00,EF,KEW GARDENS,R158,2019-01-01 04:42:21,58838861.0,67536604.0,59.0,289.0,158.50,447.5,28.8525
2019-01-01 06:00:00,EF,KEW GARDENS,R158,2019-01-01 05:31:44,58838889.0,67536630.0,2.0,0.0,240.60,147.0,0.2600
2019-01-01 07:00:00,EF,KEW GARDENS,R158,2019-01-01 05:31:44,58838889.0,67536630.0,2.0,0.0,479.20,294.0,0.2600
2019-01-01 08:00:00,EF,KEW GARDENS,R158,2019-01-01 05:31:44,58838889.0,67536630.0,2.0,0.0,717.80,441.0,0.2600


In [43]:
intterpolated_new = pd.read_pickle('../../data/cache/interpolated_turnstile.pkl.gz')

In [44]:
intterpolated_new.head()

,LINENAME,STATION,UNIT,entry_diff_abs,exit_diff_abs
datetime,,,,,
2019-01-01 00:00:00,1,103 ST,R191,1892.00,840.0
2019-01-01 00:00:00,1,103 ST,R191,1892.00,840.0
2019-01-01 01:00:00,1,103 ST,R191,1558.75,790.0
2019-01-01 02:00:00,1,103 ST,R191,1225.50,740.0
2019-01-01 03:00:00,1,103 ST,R191,892.25,690.0


In [45]:
len(intterpolated_new.STATION.unique())

380

In [23]:
results = []
sample_turnstile.groupby(['LINENAME', 'STATION', 'UNIT']).apply(lambda g: results.append(_interpolate(g)))
result = pd.concat(results)

None
None
None
None


AttributeError: 'NoneType' object has no attribute 'entry_diff_abs'

In [15]:
result[result.UNIT == "R158"].head()

,LINENAME,STATION,UNIT,datetime,ENTRIES,EXITS,entry_diff,exit_diff,entry_diff_abs,exit_diff_abs,time_diffs
datetime,,,,,,,,,,,
2019-01-01 00:18:03,EF,KEW GARDENS,R158,2019-01-01 00:18:03,58838769.0,67536224.0,184.0,333.0,184.0,333.0,25.2075
2019-01-01 00:19:32,EF,KEW GARDENS,R158,2019-01-01 00:19:32,58838773.0,67536224.0,4.0,0.0,4.0,0.0,0.2225
2019-01-01 00:20:59,EF,KEW GARDENS,R158,2019-01-01 00:20:59,58838773.0,67536224.0,0.0,0.0,0.0,0.0,0.2175
2019-01-01 00:22:23,EF,KEW GARDENS,R158,2019-01-01 00:22:23,58838773.0,67536228.0,0.0,4.0,0.0,4.0,0.2100
2019-01-01 00:43:09,EF,KEW GARDENS,R158,2019-01-01 00:43:09,58838778.0,67536239.0,5.0,11.0,5.0,11.0,3.1150


In [3]:
df_interpolated = pd.read_pickle('../../data/cache/interpolated_turnstile.pkl.gz')

In [4]:
df_interpolated.head()

,LINENAME,STATION,UNIT,entry_diff_abs,exit_diff_abs,time_diffs
datetime,,,,,,
2019-01-01 00:00:00,1,103 ST,R191,1892.00,840.0,24.0
2019-01-01 01:00:00,1,103 ST,R191,1558.75,790.0,24.0
2019-01-01 02:00:00,1,103 ST,R191,1225.50,740.0,24.0
2019-01-01 03:00:00,1,103 ST,R191,892.25,690.0,24.0
2019-01-01 04:00:00,1,103 ST,R191,559.00,640.0,24.0


In [5]:
df_interpolated[df_interpolated.UNIT == "R158"]

,LINENAME,STATION,UNIT,entry_diff_abs,exit_diff_abs,time_diffs
datetime,,,,,,


In [6]:
len(df_interpolated.STATION.unique())

370

In [8]:
el_list = pd.read_csv('../../data/interim/crosswalks/ee_turnstile.csv')

In [9]:
el_list.remote = el_list.remote.apply(lambda x: literal_eval(str(x)))
lst_col = 'remote'
subway_turnstile = pd.DataFrame({col:np.repeat(el_list[col].values, el_list[lst_col].str.len())
              for col in el_list.columns.difference([lst_col])}).assign(
                  **{lst_col:np.concatenate(el_list[lst_col].values)})[el_list.columns.tolist()]
subway_turnstile.drop(columns = ['Unnamed: 0'], inplace=True)

In [10]:
len(subway_turnstile.station_name.unique())

125

In [11]:
joined = df_interpolated.reset_index().merge(subway_turnstile,how="outer",left_on=['UNIT'],right_on=['remote'])

In [12]:
joined[joined.UNIT == "R158"].head()

,datetime,LINENAME,STATION,UNIT,entry_diff_abs,exit_diff_abs,time_diffs,station_name,equipment_id,description,borough,subway_lines,equipment_type,ada_compliant,is_active,clean_lines,remote


In [ ]:
len(joined.STATION.unique())

In [ ]:
clean_turn = pd.read_pickle('../../data/cache/cleaned_turnstile.pkl.gz')

In [ ]:
clean_turn[clean_turn.UNIT == "R002"].head()

In [ ]:
len(clean_turn.STATION.unique())

In [ ]:
filtered_turnstile = clean_turn.groupby(['LINENAME', 'STATION', 'UNIT', 'datetime']).sum().reset_index()

In [ ]:
filtered_turnstile.head()

In [ ]:
filtered_turnstile.set_index(pd.DatetimeIndex(filtered_turnstile.datetime), inplace=True)

In [ ]:
def _interpolate(data):
    augmented = data.resample('1H').asfreq()
    augmented.entry_diff_abs.interpolate(method='linear', inplace=True)
    augmented.exit_diff_abs.interpolate(method='linear', inplace=True)
    augmented.fillna(method='ffill', inplace=True)
    return augmented

In [ ]:
sample_turnstile = filtered_turnstile[filtered_turnstile.UNIT == "R191"]

In [ ]:
results = []
sample_turnstile.groupby(['LINENAME', 'STATION', 'UNIT']).apply(lambda g: results.append(_interpolate(g)))

In [ ]:
result = pd.concat(results)

In [ ]:
result.head()

In [ ]:
el_list.head()

In [ ]:
import re

In [ ]:
el_list[el_list.equipment_type=="EL"]["equipment_id"].str.contains('X').sum()

In [ ]:
len(el_list.equipment_id.unique())

In [ ]:
102/612

In [ ]:
el_list.equipment_type.unique()

In [ ]:
int_cache = pd.read_pickle('../../data/cache/interpolated_turnstile.pkl.gz')

In [ ]:
int_cache.head()

In [46]:
df = pd.read_pickle('../../data/processed/all_turnstiles_with_outage.pkl.gz')

In [47]:
df.head()

,datetime,STATION,UNIT,equipment_id,station_name,Percentage,entry_diff_abs,exit_diff_abs,Planned Outage,subway_lines
0,2019-01-01 00:00:00,103 ST,R191,NaN,NaN,0.0,1892.00,840.0,NaN,NaN
1,2019-01-01 00:00:00,103 ST,R191,NaN,NaN,0.0,1892.00,840.0,NaN,NaN
2,2019-01-01 01:00:00,103 ST,R191,NaN,NaN,0.0,1558.75,790.0,NaN,NaN
3,2019-01-01 02:00:00,103 ST,R191,NaN,NaN,0.0,1225.50,740.0,NaN,NaN
4,2019-01-01 03:00:00,103 ST,R191,NaN,NaN,0.0,892.25,690.0,NaN,NaN


In [49]:
len(df.STATION.unique())

380